### Translate
    # addsub
    # multiarith
    # singleeq

In [1]:
import openai
import yaml
from src import utils
import time
from pprint import pprint
import os
import argparse
from tqdm import tqdm
import re

In [2]:
# PROMPT = {
#     '3p' : get_3p_prompt,
#     'cot' : get_CoT_prompt,
# }

In [3]:
api = 'openai'
api_key_file = '/userhomes/philhoon/kt-ai-challenge/api-openai.yaml'

In [4]:
# model_name="gpt-3.5-turbo-0301",
model_name = "gpt-3.5-turbo-0613"

max_token=2048
temperature=0.0

In [5]:
input_file = '/userhomes/philhoon/kt-ai-challenge/eng_data/D_multiarith.json'

In [6]:
output_file_name = 'kor-' + input_file.split('/')[-1]

In [7]:
output_path = '/userhomes/philhoon/kt-ai-challenge/eng_data'
output_file = output_path + '/' + output_file_name
print(output_file)

/userhomes/philhoon/kt-ai-challenge/eng_data/kor-D_multiarith.json


In [8]:
# openai key
key_dict = utils.get_keys(api_key_file)
openai.api_key = key_dict[api]

In [9]:
# output_file = '/userhomes/philhoon/kt-ai-challenge/data/public_mwp_data_v2_reasoning.jsonl'
# print(f'output_file : {output_file}')

In [10]:
# openai-gpt-3.5-turbo-0301-2048-0.0-gsm8k-test.jsonl -> openai-gpt-3.5-turbo-0301-3p-2048-0.0-gsm8k-test.jsonl
# openai-gpt-3.5-turbo-0301-gsm8k-test.jsonl -> openai-gpt-3.5-turbo-0301-3p-inf-1.0-gsm8k-test.jsonl
# openai-gpt-3.5-turbo-0301-2048-0.0-gsm8k-test.jsonl -> openai-gpt-3.5-turbo-0301-3p-2048-0.0-gsm8k-test.jsonl

In [11]:
# output_file = output_path + '/' + 'gsm8k-korean.jsonl'
# print(output_file)

In [12]:
# Jsonlines writier
jsonl_writer = utils.JSONLWriter(output_file)

In [13]:
data = utils.read_json_file(input_file)

In [14]:
from pprint import pprint

In [15]:
data = data['data']

In [16]:
def remove_line_split(context):
    context = re.sub(r'\n{1,}', ' ', context)
    context = re.sub(r'\s{2,}', ' ', context)
    return context

In [17]:
def generate_answer(message, model_name, max_token, temperature):
    """
    Generate response
    """
    try:
        completion = openai.ChatCompletion.create(
            model=model_name,
            messages=message,
            temperature=temperature,
            max_tokens=max_token,
            n=1)
    except:
        print("retrying due to an error......")
        time.sleep(20)
        return generate_answer(message, model_name, max_token, temperature)

    return completion

In [18]:
def get_trans_msg(context):
    """
    Message for chat api
    """

    msg = [
        {
            "role" : "system",
            "content" : "Translate given setences into Korean."
        },
        {
            "role" : "user",
            "content" : context
        }
    ]
    
    return msg

In [19]:
from tqdm import tqdm

In [ ]:
cnt = 0
for ins in tqdm(data.values()):
    instance = ins[0]

#     pprint(cur_ins)
    
    eng_comp = remove_line_split(instance['reasoning_completion'])
    eng_query = instance['question']
    
    query_msg = get_trans_msg(eng_query)
    ans_msg = get_trans_msg(eng_comp)
    
    query_completion = generate_answer(query_msg, model_name, max_token, temperature)
    q_response = query_completion["choices"][0]["message"]["content"]
    
    ans_completion = generate_answer(ans_msg, model_name, max_token, temperature)
    ans_response = ans_completion["choices"][0]["message"]["content"]
    
    instance['kor_query'] = q_response
    instance['kor_ans'] = ans_response
    pprint(instance)
#     jsonl_writer.write_json_line(ins)a


  0%|▍                                                                                                                                                                                                                                    | 1/600 [00:09<1:32:48,  9.30s/it]

{'answer': '39',
 'completion': ' 39.',
 'completion_index': 0,
 'kor_ans': '데비와 그녀의 여동생은 총 32 + 42 = 74개의 사탕을 가지고 있습니다. 그들은 35개를 먹었으므로, 남은 '
            '사탕은 74 - 35 = 39개입니다.',
 'kor_query': '할로윈에 데비와 그녀의 여동생은 받은 사탕을 합쳤습니다. 데비는 32개의 사탕을 가지고 있었고, 그녀의 여동생은 '
              '42개를 가지고 있었습니다. 첫날 밤에 35개의 사탕을 먹었다면, 남은 사탕은 몇 개인가요?',
 'prompt': 'Q: For Halloween Debby and her sister combined the candy they '
           'received. Debby had 32 pieces of candy while her sister had 42. If '
           'they ate 35 pieces the first night, how many pieces do they have '
           'left?\n'
           "A: Let's think step by step. \n"
           '\n'
           'Debby and her sister have a total of 32 + 42 = 74 pieces of '
           'candy.\n'
           'They eat 35 pieces, so they have 74 - 35 = 39 pieces left. '
           'Therefore, the answer (arabic numerals) is',
 'question': 'For Halloween Debby and her sister combined the candy they '
             'received. Debby had 32 pieces of candy 